In [1]:
from engine import RecommendationEngine
from pyspark.sql import SparkSession
import flask
import os 

In [2]:
spark = SparkSession.builder \
           .appName('Web App') \
           .config("spark.jars", "mysql-connector-j-8.0.32.jar")\
           .config("spark.driver.memory", "6g") \
           .config("spark.executor.memory", "4g") \
           .getOrCreate()

In [3]:
path = os.getcwd()

In [4]:
movies_detail_data =  spark.read.format("jdbc").option("driver","com.mysql.cj.jdbc.Driver") \
                                .option("url", "jdbc:mysql://cap2-database/MovieLens").option("dbtable", "movies_detail") \
                                .option("user", "root").option("password", "123").load().rdd.cache()

In [5]:
engine = RecommendationEngine(spark)

INFO:engine:Starting up the Recommendation Engine: 
INFO:engine:Loading Movies data...
INFO:engine:Loading Ratings data...
INFO:engine:Loading Users data...
INFO:engine:Counting how many ratings per movie...
INFO:engine:Training the ALS model...
INFO:engine:ALS model built!


In [ ]:
app = flask.Flask(__name__, template_folder = os.path.join(path, 'template'))
@app.route('/', methods = ['GET', 'POST'])
def main():
    if flask.request.method == 'GET':
        return flask.render_template('index.html')
            
    if flask.request.method == 'POST':
        user_id = flask.request.form['user_id']
        results = engine.get_top_ratings(int(user_id))
        recommendations = []
        for result in results: 
            recommendation = []
            movieId = result[0]
            rating = result[1]
            count = result[2]
            details = movies_detail_data.filter(lambda x: x[0] == movieId).collect()
            for detail in details :
                recommendation.append(detail)
            recommendation.append(rating)
            recommendation.append(count)
            recommendations.append(recommendation)
        return flask.render_template('result.html', userId = user_id, recommendations = recommendations)

if __name__ == '__main__':
    app.run(host="0.0.0.0")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.19.0.3:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:172.19.0.1 - - [25/Apr/2023 03:05:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:172.19.0.1 - - [25/Apr/2023 03:06:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:172.19.0.1 - - [25/Apr/2023 03:06:34] "POST / HTTP/1.1" 200 -
INFO:py4j.clientserver:Closing down clientserver connection
INFO:werkzeug:172.19.0.1 - - [25/Apr/2023 03:07:09] "POST / HTTP/1.1" 200 -
INFO:py4j.clientserver:Closing down clientserver connection
INFO:werkzeug:172.19.0.1 - - [25/Apr/2023 03:08:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:172.19.0.1 - - [25/Apr/2023 03:08:13] "POST / HTTP/1.1" 200 -
INFO:py4j.clientserver:Closing down clientserver connection
INFO:werkzeug:172.19.0.1 - - [25/Apr/2023 03:09:11] "GET / HTTP/1.1" 200 -
IN